In [1]:
from google.colab import drive
drive.mount('/content/drive/')
import os
path="/content/drive/MyDrive/Colab Notebooks/HW4&Final/FINAL"
os.chdir(path)
os.listdir(path)
import os
print(os.getcwd())

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
/content/drive/MyDrive/Colab Notebooks/HW4&Final/FINAL


In [2]:
!pip install accelerate
!pip install transformers


In [3]:
import pandas as pd

# 假设 data_path 是您数据集的路径
data_path = "AnonymizedClinicalAbbreviationsAndAcronymsDataSet.txt"

try:
    # 尝试使用 ISO-8859-1 编码读取数据
    data = pd.read_csv(data_path, sep='|', header=None, names=[
        "Short_form", "Long_form", "Abbreviation", "Start_pos", "End_pos", "Section_info", "Context"
    ], encoding='ISO-8859-1')
except UnicodeDecodeError:
    # 如果 ISO-8859-1 编码失败，则忽略错误的字符
    data = pd.read_csv(data_path, sep='|', header=None, names=[
        "Short_form", "Long_form", "Abbreviation", "Start_pos", "End_pos", "Section_info", "Context"
    ], encoding='ISO-8859-1', errors='ignore')

print(data.head())
print(data.info())

  Short_form Long_form Abbreviation  Start_pos  End_pos  \
0         AB  abortion          AB.      231.0    233.0   
1         AB  abortion          AB.      249.0    251.0   
2         AB  abortion           AB      223.0    224.0   
3         AB  abortion          AB.      194.0    196.0   
4         AB  abortion           AB      114.0    115.0   

                     Section_info  \
0                             NaN   
1                             NaN   
2                 PAST OB HISTORY   
3  HISTORY OF THE PRESENT ILLNESS   
4             PAST OB-GYN HISTORY   

                                             Context  
0  _%#NAME#%_ _%#NAME#%_ is a 29-year-old gravida...  
1  She is now bleeding quite heavily. Ultrasound ...  
2  ALLERGIES: Heparin and Imitrex. PAST OB HISTOR...  
3  She had a pelvic ultrasound at Park Nicollet o...  
4  On _%#MMDD2007#%_, normal anatomy with anterio...  
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37500 entries, 0 to 37499
Data columns (to

In [16]:
# 假设 data 是 pandas DataFrame
labels = data['Long_form'].tolist()  # 从 DataFrame 中提取标签列
texts = data['Context'].tolist()  # 从 DataFrame 中提取文本列

# 创建从类别名称到整数索引的映射
label_list = sorted(set(labels))  # 创建一个包含所有唯一标签的列表
label_map = {label: idx for idx, label in enumerate(label_list)}

# 创建从整数索引到类别名称的逆映射
inverse_label_map = {idx: label for label, idx in label_map.items()}

# 打印标签映射以确保其正确性
print("Label Map:", label_map)
print("Inverse Label Map:", inverse_label_map)




print("Total labels:", len(label_map))
print("Total Inverse Label:", len(inverse_label_map))



# 将字符串标签转换为整数索引
labels_indexed = [label_map[label] for label in labels]

# 使用 train_test_split 分割数据集
from sklearn.model_selection import train_test_split
texts_train, texts_test, labels_train, labels_test = train_test_split(
    texts, labels_indexed, test_size=0.2, random_state=42)

Label Map: {'(PO Box) C4': 0, '(class) IA': 1, '(complement) component 3': 2, '(complement) component 4': 3, '(cycle) IB': 4, '(device) MP': 5, '(device) PD': 6, '(diltiazem) DC': 7, '(drug) AC': 8, '(drug) DC': 9, '(drug) DT': 10, '(drug) ES': 11, '(drug) IT': 12, '(drug) MP': 13, '(drug) MS': 14, '(drug) NP': 15, '(drug) OR': 16, '(drug) PAC': 17, '(drug) PD': 18, '(drug) PR': 19, '(drug) RT': 20, '(grade) IA': 21, '(grade) IB': 22, '(stage) C3': 23, '(stage) IA': 24, '(stage) IB': 25, '(status) IA': 26, '(status) IB': 27, '(type) IA': 28, '(type) IB': 29, 'American Society of Anesthesiologists': 30, 'American Society of Anesthesiologists:ASA': 31, 'BK (virus)': 32, 'C-reactive': 33, 'California': 34, "Children's Depression Inventory": 35, 'District of Columbia': 36, 'Fairview Southdale Hospital': 37, 'GENERAL ENGLISH': 38, 'Iowa': 39, 'Los Angeles': 40, 'Louisiana': 41, 'MISTAKE:EZ PAP': 42, 'MISTAKE:Oncotype DX': 43, 'MISTAKE:abduction': 44, 'Mall of America:MOA': 45, 'NAME': 46, '

In [5]:


# 显示一些基本信息以确保数据正确处理
print(f"Number of training samples: {len(texts_train)}")
print(f"Number of test samples: {len(texts_test)}")
print("Example of processed data:")
print(texts_train[0], labels_train[0])


Number of training samples: 30000
Number of test samples: 7500
Example of processed data:
Will get his home medications and continue those. Will ask neurology to review his antiepileptic regimen and have spoken with Dr. _%#NAME#%_. We will have OT and PT evaluate the patient. Suspect that he may need a TCU versus home with additional services. LABORATORY DATA: That was performed here includes the following laboratory tests. 267


In [6]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from torch.utils.data import DataLoader, Dataset
import torch

class ClinicalTextDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

# 加载预训练的Tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')


# 在将数据传给tokenizer之前，确保所有文本都是字符串
texts_train = [str(text) for text in texts_train]
texts_test = [str(text) for text in texts_test]

# 现在尝试编码
train_encodings = tokenizer(texts_train, truncation=True, padding=True, max_length=128)
test_encodings = tokenizer(texts_test, truncation=True, padding=True, max_length=128)


# 编码数据
train_encodings = tokenizer(texts_train, truncation=True, padding=True, max_length=128)
test_encodings = tokenizer(texts_test, truncation=True, padding=True, max_length=128)

# 创建数据集
train_dataset = ClinicalTextDataset(train_encodings, labels_train)
test_dataset = ClinicalTextDataset(test_encodings, labels_test)

# 加载预训练的BERT模型，添加一个顶层用于分类
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(label_map))

# 定义训练参数
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=1,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=100,
)

# 初始化 Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

# 训练模型
trainer.train()


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss
100,5.888900
200,5.656900
300,5.250900
400,4.905200
500,4.635800
600,4.260000
700,3.746200
800,3.317800
900,2.845000
1000,2.443900


TrainOutput(global_step=3750, training_loss=1.8311553319295248, metrics={'train_runtime': 242.12, 'train_samples_per_second': 123.905, 'train_steps_per_second': 15.488, 'total_flos': 1979516413440000.0, 'train_loss': 1.8311553319295248, 'epoch': 1.0})

In [ ]:
# 保存训练好的模型
model_path = "./biobert_finetuned"
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

In [18]:
# 确保模型的分类头设置正确
print("Expected number of labels (from label map):", len(label_map))
model_config = model.config  # 假设模型有一个可访问的配置属性
print("Number of labels in model configuration:", model_config.num_labels)




Expected number of labels (from label map): 351
Number of labels in model configuration: 351


In [19]:
# 进行一次快速测试以验证模型输出和逆映射
from torch.utils.data import DataLoader

# 假设已经定义了一个数据加载器
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

for batch in test_loader:
    batch = {k: v.to(model.device) for k, v in batch.items()}
    with torch.no_grad():
        outputs = model(**batch)
    pred_indices = outputs.logits.argmax(-1).tolist()
    print("Predicted indices:", pred_indices)  # 查看预测索引
    predicted_labels = [inverse_label_map.get(idx, 'Label Index Missing') for idx in pred_indices]
    actual_labels = [inverse_label_map.get(label.item(), 'Label Index Missing') for label in batch['labels']]
    for pred, actual in zip(predicted_labels, actual_labels):
        print(f"Predicted: {pred}, Actual: {actual}")

Predicted indices: [47]
Predicted: Narcotics Anonymous, Actual: Narcotics Anonymous
Predicted indices: [179]
Predicted: idiopathic thrombocytopenic purpura, Actual: idiopathic thrombocytopenic purpura
Predicted indices: [82]
Predicted: aortic valve replacement, Actual: augmented voltage right arm
Predicted indices: [47]
Predicted: Narcotics Anonymous, Actual: Narcotics Anonymous
Predicted indices: [305]
Predicted: respiratory therapy, Actual: respiratory therapy
Predicted indices: [62]
Predicted: acetylsalicylic acid, Actual: acetylsalicylic acid
Predicted indices: [2]
Predicted: (complement) component 3, Actual: (complement) component 3
Predicted indices: [52]
Predicted: T2 (MRI), Actual: T2 (MRI)
Predicted indices: [235]
Predicted: multiples of median, Actual: multiples of median
Predicted indices: [8]
Predicted: (drug) AC, Actual: (drug) AC
Predicted indices: [32]
Predicted: BK (virus), Actual: BK (virus)
Predicted indices: [267]
Predicted: physical therapy, Actual: physical therapy

KeyboardInterrupt: 